## Censo de Población y Viviendas
Descargar CPV 2020 Aguascalientes
https://www.inegi.org.mx/programas/ccpv/2020/#datos_abiertos

In [ ]:
import json
import requests
import os
import zipfile
import geopandas as gpd 
import pandas as pd
import io
import contextily as cx

In [ ]:
url= "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/ageb_mza_urbana_19_cpv2020_csv.zip"

In [ ]:
response = requests.get(url)
if response.status_code == 200:
    # Crear un archivo ZIP en memoria
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Extraer los archivos del ZIP
        z.extractall('CPV_2020')
else:
    print('Error al descargar el archivo ZIP')

In [ ]:
df = pd.read_csv('CPV_2020/ageb_mza_urbana_19_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_19_cpv2020.csv')
df.head()

In [ ]:
urlMza = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/889463807469_s.zip"
response = requests.get(urlMza)
if response.status_code == 200:
    # Crear un archivo ZIP en memoria
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Extraer los archivos del ZIP
        z.extractall('Manzanas_2020')
else:
    print('Error al descargar el archivo')

In [ ]:
gdf= gpd.read_file("Manzanas_2020/19_aguascalientes.zip!/conjunto_de_datos/31m.shp").to_crs(4326)
gdf.head()

In [ ]:
df['CVEGEO'] = df['ENTIDAD'].map(str).str.zfill(2) + df['MUN'].map(str).str.zfill(3) + df['LOC'].map(str).str.zfill(4) + df['AGEB'].map(str).str.zfill(4) + df['MZA'].map(str).str.zfill(3)

In [ ]:
gdf_merged = gpd.GeoDataFrame(pd.merge(gdf, df, on='CVEGEO'))

In [ ]:
gdf_filtrado = gdf_merged.query("NOM_LOC == 'Aguascalientes'")

In [ ]:
gdf_filtrado = gdf_filtrado[gdf_filtrado['PCDISC_MOT'] != '*']
gdf_filtrado['PCDISC_MOT'] = gdf_filtrado['PCDISC_MOT'].astype(int)

In [ ]:
ax = gdf_filtrado.plot("PCDISC_MOT", cmap="tab10", legend=True, figsize=(14, 8))
cx.add_basemap(ax, crs=gdf_filtrado.crs.to_string())